In [0]:
from IPython.display import display,HTML
def dhtml(str):
    display(HTML("""<style>
    @import url('https://fonts.googleapis.com/css?family=Ewert&effect=3d');      
    </style><h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>
    %s</h1>"""%str))

In [2]:
dhtml('Code Modules & Helpful Functions')

In [3]:
!pip install --upgrade neural_structured_learning --user

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
path='/root/.local/lib/python3.6/site-packages'
import sys; sys.path.append(path)
import warnings; warnings.filterwarnings('ignore')
import h5py,urllib,zipfile
import pylab as pl,pandas as pd
import numpy as np,tensorflow as tf
import neural_structured_learning as nsl
from sklearn.model_selection import train_test_split

In [0]:
fpath='https://olgabelitskaya.github.io/'
def prepro(x_train,y_train,x_test,y_test,n_class):
    n=int(len(x_test)/2)    
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    cy_train=tf.keras.utils.to_categorical(y_train,n_class) 
    cy_valid=tf.keras.utils.to_categorical(y_valid,n_class)
    cy_test=tf.keras.utils.to_categorical(y_test,n_class)
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape],
                     [cy_train.shape,cy_valid.shape,cy_test.shape]],
                    columns=['train','valid','test'],
                    index=['images','labels','encoded labels'])
    display(df)
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test],
            [cy_train,cy_valid,cy_test]]

In [6]:
dhtml('Data Loading & Preprocessing')

In [7]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1],
 [cy_train1,cy_valid1,cy_test1]]=\
prepro(x_train1/255,y_train1.reshape(-1,1),
       x_test1/255,y_test1.reshape(-1,1),10)

11493376/11490434 [==============================] - 0s 0us/step


,train,valid,test
images,"(60000, 28, 28)","(5000, 28, 28)","(5000, 28, 28)"
labels,"(60000, 1)","(5000, 1)","(5000, 1)"
encoded labels,"(60000, 10)","(5000, 10)","(5000, 10)"


In [8]:
(x_train2,y_train2),(x_test2,y_test2)=\
tf.keras.datasets.cifar10.load_data()
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2],
 [cy_train2,cy_valid2,cy_test2]]=\
prepro(x_train2/255,y_train2,x_test2/255,y_test2,10)

170500096/170498071 [==============================] - 2s 0us/step


,train,valid,test
images,"(50000, 32, 32, 3)","(5000, 32, 32, 3)","(5000, 32, 32, 3)"
labels,"(50000, 1)","(5000, 1)","(5000, 1)"
encoded labels,"(50000, 10)","(5000, 10)","(5000, 10)"


In [9]:
zf='LetterColorImages_123.h5.zip'
input_file=urllib.request.urlopen(fpath+zf)
output_file=open(zf,'wb'); 
output_file.write(input_file.read())
output_file.close(); input_file.close()
zipf=zipfile.ZipFile(zf,'r')
zipf.extractall(''); zipf.close()
f=h5py.File(zf[:-4],'r')
keys=list(f.keys()); keys
images=np.array(f[keys[1]])/255
labels=np.array(f[keys[2]]).astype('int').reshape(-1,1)-1
x_train3,x_test3,y_train3,y_test3=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train3,x_valid3,x_test3],
 [y_train3,y_valid3,y_test3],
 [cy_train3,cy_valid3,cy_test3]]=\
prepro(x_train3,y_train3,x_test3,y_test3,33)

,train,valid,test
images,"(11352, 32, 32, 3)","(1419, 32, 32, 3)","(1419, 32, 32, 3)"
labels,"(11352, 1)","(1419, 1)","(1419, 1)"
encoded labels,"(11352, 33)","(1419, 33)","(1419, 33)"


In [10]:
zf='FlowerColorImages.h5.zip'
input_file=urllib.request.urlopen(fpath+zf)
output_file=open(zf,'wb'); 
output_file.write(input_file.read())
output_file.close(); input_file.close()
zipf=zipfile.ZipFile(zf,'r')
zipf.extractall(''); zipf.close()
f=h5py.File(zf[:-4],'r')
keys=list(f.keys())
images=np.array(f[keys[0]])/255
labels=np.array(f[keys[1]]).astype('int').reshape(-1,1)
x_train4,x_test4,y_train4,y_test4=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train4,x_valid4,x_test4],
 [y_train4,y_valid4,y_test4],
 [cy_train4,cy_valid4,cy_test4]]=\
prepro(x_train4,y_train4,x_test4,y_test4,10)

,train,valid,test
images,"(168, 128, 128, 3)","(21, 128, 128, 3)","(21, 128, 128, 3)"
labels,"(168, 1)","(21, 1)","(21, 1)"
encoded labels,"(168, 10)","(21, 10)","(21, 10)"


In [0]:
dhtml('Models with Adversarial Regularization')
dhtml('MLP Base')

In [0]:
batch_size=64; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size),name='input'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(n_class,activation=tf.nn.softmax)
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1,'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1,'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/7

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

938/938 - 9s - loss: 0.2699 - sparse_categorical_crossentropy: 0.2174 - sparse_categorical_accuracy: 0.9354 - adversarial_loss: 0.2628 - val_loss: 0.2566 - val_sparse_categorical_crossentropy: 0.2066 - val_sparse_categorical_accuracy: 0.9351 - val_adversarial_loss: 0.2500
Epoch 2/7
938/938 - 8s - loss: 0.1141 - sparse_categorical_crossentropy: 0.0898 - sparse_categorical_accuracy: 0.9723 - adversarial_loss: 0.1212 - val_loss: 0.1824 - val_sparse_categorical_crossentropy: 0.1467 - val_sparse_ca

In [0]:
adv_model.evaluate({'input':x_test1,'label':y_test1})

157/157 [==============================] - 1s 4ms/step - loss: 0.0745 - sparse_categorical_crossentropy: 0.0590 - sparse_categorical_accuracy: 0.9840 - adversarial_loss: 0.0774


[0.07448980957269669,
 0.05901719257235527,
 0.984000027179718,
 0.0773630291223526]

In [0]:
batch_size=64; img_size=32; n_class=10; epochs=10
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

782/782 - 35s - loss: 2.1229 - sparse_categorical_crossentropy: 1.7573 - sparse_categorical_accuracy: 0.3839 - adversarial_loss: 1.8282 - val_loss: 2.1685 - val_sparse_categorical_crossentropy: 1.7982 - val_sparse_categorical_accuracy: 0.3640 - val_adversarial_loss: 1.8515
Epoch 2/10
782/782 - 36s - loss: 1.7820 - sparse_categorical_crossentropy: 1.4750 - sparse_categorical_accuracy: 0.4757 - adversarial_loss: 1.5351 - val_loss: 1.9264 - val_sparse_categorical_crossentropy: 1.5946 - val_spars

In [0]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

157/157 [==============================] - 2s 13ms/step - loss: 3.6545 - sparse_categorical_crossentropy: 3.0013 - sparse_categorical_accuracy: 0.3900 - adversarial_loss: 3.2659


[3.654510736465454, 3.001328468322754, 0.38999998569488525, 3.2659106254577637]

In [0]:
batch_size=128; img_size=32; n_class=33; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(33,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/100
89/89 - 7s - loss: 3.3708 - sparse_categorical_crossentropy: 2.7465 - sparse_categorical_accuracy: 0.2308 - adversarial_loss: 3.1213 - val_loss: 6.1906 - val_sparse_categorical_crossentropy: 5.1431 - val_sparse_categorical_accuracy: 0.0284 - val_adversarial_loss: 5.2376
Epoch 2/100
89/89 - 7s - loss: 2.3859 - sparse_categorical_crossentropy: 1.9202 - sparse_categorical_accuracy: 0.4326 - adversarial_loss: 2.3288 - val_loss: 6.1017 - val_sparse_categorical_crossentropy: 5.0369 - val_sparse_categorical_accuracy: 0.0959 - val_adversarial_loss: 5.3241
Epoch 3/100
89/89 - 7s - loss: 1.9453 - sparse_categorical_crossentropy: 1.5532 - sparse_categorical_accuracy: 0.5351 - adversarial_loss: 1.9605 - val_loss: 5.1579 - val_sparse_categorical_crossentropy: 4.2392 - val_sparse_categorical_accuracy: 0.1555 - val_adversarial_loss: 4.5934
Epoch 4/100
89/89 - 7s - loss: 1.6933 - sparse_categorical_crossentropy: 1.3420 - sparse_categorical_accuracy: 0.5919 - adversarial_loss: 1.7566 - val_

In [0]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

45/45 [==============================] - 1s 23ms/step - loss: 2.3239 - sparse_categorical_crossentropy: 1.7559 - sparse_categorical_accuracy: 0.7301 - adversarial_loss: 2.8400


[2.323880910873413, 1.7558883428573608, 0.7300916314125061, 2.8399624824523926]

In [0]:
batch_size=16; img_size=128; n_class=10; epochs=50
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/50
11/11 - 1s - loss: 3.4688 - sparse_categorical_crossentropy: 2.8753 - sparse_categorical_accuracy: 0.2440 - adversarial_loss: 2.9678 - val_loss: 4.4298 - val_sparse_categorical_crossentropy: 3.6893 - val_sparse_categorical_accuracy: 0.1875 - val_adversarial_loss: 3.7027
Epoch 2/50
11/11 - 1s - loss: 1.5130 - sparse_categorical_crossentropy: 1.2569 - sparse_categorical_accuracy: 0.6190 - adversarial_loss: 1.2806 - val_loss: 3.8903 - val_sparse_categorical_crossentropy: 3.2399 - val_sparse_categorical_accuracy: 0.3125 - val_adversarial_loss: 3.2519
Epoch 3/50
11/11 - 1s - loss: 0.4232 - sparse_categorical_crossentropy: 0.3509 - sparse_categorical_accuracy: 0.9107 - adversarial_loss: 0.3617 - val_loss: 4.8662 - val_sparse_categorical_crossentropy: 4.0525 - val_sparse_categorical_accuracy: 0.1875 - val_adversarial_loss: 4.0688
Epoch 4/50
11/11 - 1s - loss: 0.0832 - sparse_categorical_crossentropy: 0.0685 - sparse_categorical_accuracy: 0.9821 - adversarial_loss: 0.0733 - val_loss

In [0]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

1/1 [==============================] - 0s 1ms/step - loss: 4.0883 - sparse_categorical_crossentropy: 3.3562 - sparse_categorical_accuracy: 0.4762 - adversarial_loss: 3.6602


[4.088286399841309, 3.3562381267547607, 0.4761904776096344, 3.6602420806884766]

In [0]:
dhtml('CNN Base')

In [0]:
batch_size=128; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,1),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')    
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1.reshape(x_train1.shape[0],28,28,1),
     'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1.reshape(x_valid1.shape[0],28,28,1),
     'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/7

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

469/469 - 356s - loss: 0.4222 - sparse_categorical_crossentropy: 0.3495 - sparse_categorical_accuracy: 0.8890 - adversarial_loss: 0.3632 - val_loss: 0.1553 - val_sparse_categorical_crossentropy: 0.1269 - val_sparse_categorical_accuracy: 0.9587 - val_adversarial_loss: 0.1420
Epoch 2/7
469/469 - 353s - loss: 0.1064 - sparse_categorical_crossentropy: 0.0870 - sparse_categorical_accuracy: 0.9746 - adversarial_loss: 0.0966 - val_loss: 0.0676 - val_sparse_categorical_crossentropy: 0.0549 - val_spars

In [0]:
adv_model.evaluate({'input':x_test1.reshape(x_test1.shape[0],28,28,1),
                    'label':y_test1})

157/157 [==============================] - 15s 96ms/step - loss: 0.0187 - sparse_categorical_crossentropy: 0.0150 - sparse_categorical_accuracy: 0.9950 - adversarial_loss: 0.0185


[0.018723322078585625,
 0.015017231926321983,
 0.9950000047683716,
 0.01853051595389843]

In [0]:
batch_size=64; img_size=32; n_class=10; epochs=10
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(96,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

782/782 - 284s - loss: 2.0013 - sparse_categorical_crossentropy: 1.6629 - sparse_categorical_accuracy: 0.3870 - adversarial_loss: 1.6922 - val_loss: 1.6769 - val_sparse_categorical_crossentropy: 1.3874 - val_sparse_categorical_accuracy: 0.5122 - val_adversarial_loss: 1.4475
Epoch 2/10
782/782 - 285s - loss: 1.5916 - sparse_categorical_crossentropy: 1.3187 - sparse_categorical_accuracy: 0.5262 - adversarial_loss: 1.3642 - val_loss: 1.4699 - val_sparse_categorical_crossentropy: 1.2113 - val_spa

In [0]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

157/157 [==============================] - 15s 95ms/step - loss: 1.2206 - sparse_categorical_crossentropy: 0.9919 - sparse_categorical_accuracy: 0.6584 - adversarial_loss: 1.1431


[1.2205510139465332,
 0.9919253587722778,
 0.6583999991416931,
 1.1431292295455933]

In [0]:
batch_size=128; img_size=32; n_class=33; epochs=200
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(33,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/200
89/89 - 94s - loss: 4.1673 - sparse_categorical_crossentropy: 3.4723 - sparse_categorical_accuracy: 0.0398 - adversarial_loss: 3.4751 - val_loss: 4.1066 - val_sparse_categorical_crossentropy: 3.4203 - val_sparse_categorical_accuracy: 0.0675 - val_adversarial_loss: 3.4315
Epoch 2/200
89/89 - 94s - loss: 4.0173 - sparse_categorical_crossentropy: 3.3446 - sparse_categorical_accuracy: 0.0654 - adversarial_loss: 3.3638 - val_loss: 3.9495 - val_sparse_categorical_crossentropy: 3.2871 - val_sparse_categorical_accuracy: 0.0831 - val_adversarial_loss: 3.3120
Epoch 3/200
89/89 - 93s - loss: 3.8783 - sparse_categorical_crossentropy: 3.2260 - sparse_categorical_accuracy: 0.0798 - adversarial_loss: 3.2615 - val_loss: 3.8477 - val_sparse_categorical_crossentropy: 3.2008 - val_sparse_categorical_accuracy: 0.0952 - val_adversarial_loss: 3.2345
Epoch 4/200
89/89 - 93s - loss: 3.7906 - sparse_categorical_crossentropy: 3.1527 - sparse_categorical_accuracy: 0.0908 - adversarial_loss: 3.1899 - 

In [0]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

45/45 [==============================] - 6s 131ms/step - loss: 0.3680 - sparse_categorical_crossentropy: 0.2585 - sparse_categorical_accuracy: 0.9295 - adversarial_loss: 0.5477


[0.3680289685726166,
 0.2584863603115082,
 0.9295278191566467,
 0.5477129817008972]

In [0]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [0]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Epoch 1/100
11/11 - 20s - loss: 2.7979 - sparse_categorical_crossentropy: 2.3331 - sparse_categorical_accuracy: 0.1369 - adversarial_loss: 2.3240 - val_loss: 2.7506 - val_sparse_categorical_crossentropy: 2.2915 - val_sparse_categorical_accuracy: 0.1875 - val_adversarial_loss: 2.2954
Epoch 2/100
11/11 - 20s - loss: 2.5536 - sparse_categorical_crossentropy: 2.1291 - sparse_categorical_accuracy: 0.2976 - adversarial_loss: 2.1227 - val_loss: 2.5459 - val_sparse_categorical_crossentropy: 2.1207 - val_sparse_categorical_accuracy: 0.2500 - val_adversarial_loss: 2.1259
Epoch 3/100
11/11 - 20s - loss: 2.1390 - sparse_categorical_crossentropy: 1.7798 - sparse_categorical_accuracy: 0.4048 - adversarial_loss: 1.7962 - val_loss: 2.2541 - val_sparse_categorical_crossentropy: 1.8767 - val_sparse_categorical_accuracy: 0.3750 - val_adversarial_loss: 1.8871
Epoch 4/100
11/11 - 20s - loss: 1.7899 - sparse_categorical_crossentropy: 1.4852 - sparse_categorical_accuracy: 0.4464 - adversarial_loss: 1.5236 - 

In [0]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

1/1 [==============================] - 0s 2ms/step - loss: 1.2200 - sparse_categorical_crossentropy: 1.0023 - sparse_categorical_accuracy: 0.7143 - adversarial_loss: 1.0889


[1.2200490236282349,
 1.0022777318954468,
 0.7142857313156128,
 1.0888566970825195]